<a href="https://colab.research.google.com/github/junhyeongShin/colab/blob/main/%EB%B2%A0%EC%9D%B4%EC%8A%A4%EC%BD%94%EB%93%9C_to_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#텐서 차원 변경 할때 사용되는 메소드
# torch.Size([4, 3, 32, 32])
# train_t = np.transpose(train, (0, 2, 3, 1))
# train_t.shape
# torch.Size([4, 32, 32, 3])

In [ ]:
# torch 임포트 다운로드

pip install torch

In [ ]:
pip install torchvision

In [ ]:
pip install Pillow

In [ ]:

import datetime
import os
import time

import numpy as np
import pandas as pd


from matplotlib import pyplot as plt
from pathlib import Path

In [ ]:
# 구글 드라이브에 연결
from google.colab import drive
drive.mount('/content/drive')
# 해당폴더로 이동
%cd drive/My\ Drive/Colab Notebooks

In [ ]:
data_path = './data_v2/'
train_data_path = os.path.join(data_path, "train_data_v2")
print(train_data_path)
file_list = os.listdir(train_data_path)
file_list.sort()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchvision import transforms, utils
from torchsummary import summary

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm

device = None
# gpu 사용
if torch.cuda.is_available() :
    device = torch.device('cuda')
else : 
    device = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', device)

In [ ]:
path = data_path
train_csv = pd.read_csv(path+'train_v2.csv')
train_csv.head()

In [ ]:
train_path = sorted(glob(path+'train_data_v2/*.npy'))
train_path = train_path[-30*12:]

train = []
for p in tqdm(train_path):
    # train.append(np.load(p))
    tmp = np.load(p)
    # tmp[:,:,0] = tmp[:,:,0]/250
    train.append(tmp)
    # (360, 448, 304, 5)
train = np.array(train)
train.shape

In [ ]:
train_month = []
for i in range(12):
    train_month.append(train[i::12, :,:])
train_month = np.array(train_month)

train_month.shape

In [ ]:
stride = 64
temp = np.zeros([train_month.shape[0],
                 train_month.shape[1],
                 train_month.shape[2]+(stride-train_month.shape[2]%stride),
                 train_month.shape[3]+(stride-train_month.shape[3]%stride),
                 train_month.shape[4]], np.uint8)
temp[:, :,:train_month.shape[2], :train_month.shape[3], :] = train_month
train_month = temp
del temp

In [ ]:
window_size = 5

In [ ]:
x_train = []
y_train = []

for i in range(train_month.shape[1]-window_size):
    x_train.append(train_month[:, i:i+window_size, :,:,:])
    y_train.append(train_month[:, i+window_size:i+window_size+1, :,:,:])
    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
#stride = 32
feature_size = 128
x_train_ = []
y_train_ = []
for y in range(x_train.shape[0]):
    for m in range(12):
        for i in range((x_train.shape[3]-feature_size)//stride+1):
            for j in range((x_train.shape[4]-feature_size)//stride+1):
                x_ = x_train[y, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
                y_ = y_train[y, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
                x_train_.append(x_)
                y_train_.append(y_)

x_train_ = np.array(x_train_)
y_train_ = np.array(y_train_)

x_val_ = []
y_val_ = []

for m in range(12):
    for i in range((x_train.shape[3]-feature_size)//stride+1):
        for j in range((x_train.shape[4]-feature_size)//stride+1):
            x_ = x_train[-1, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
            y_ = y_train[-1, m, :, stride*i:stride*i+feature_size, stride*j:stride*j+feature_size, :1]
            x_val_.append(x_)
            y_val_.append(y_)
            
x_val_ = np.array(x_val_)
y_val_ = np.array(y_val_)

In [ ]:
x_train_.shape, y_train_.shape, x_val_.shape, y_val_.shape

In [ ]:
# 데이터 이미지 확인
i = 38
plt.figure(figsize=(15,10))
for y in range(x_train_.shape[1]):
    plt.subplot(1,window_size,y+1)
    plt.imshow(x_train_[i,y,:,:,0])
plt.show()
plt.imshow(y_train_[i,-1,:,:,0])

In [ ]:
# 데이터 셋 구성부분입니다.

class TrainDataset(Dataset):
    def __init__(self, train_x,train_y):
        super(TrainDataset, self).__init__()

        self.train_x = train_x
        self.train_y = train_y
        # self.transform = transform

    def __len__(self):
        # 데이터 갯수
        return self.train_x.shape[0]
    

    # 데이터를 들고올때 이미지를 잘라서 필요한 갯수만큼 들고올수 있도록 설정해 놓았기때문에
    # 해당 부분에서는 넘파이를 탠서로 변경해주고있습니다.
    def __getitem__(self, idx):

        x = self.train_x[idx]
        y = self.train_y[idx]

        #torch를 사용해서 탠서로 변경해줍니다. 탠서의 자료형은 float입니다.
        x = torch.tensor(x)
        x = torch.FloatTensor(x/250)

        # 모델에 넣어줄때 필요한 순서를 맞추기 위해서
        # 탠서 차원 순서를 변경해주고 있습니다.
        # ex) y = x.permute (1,0) => y의 1번째 차원에 x의 0번째 차원으로 변경하겠다.
        x = x.permute(0,3,2,1)
        x = x.permute(1,0,2,3)
        # 5, 128, 128, 1

        y = torch.tensor(y)
        y = torch.FloatTensor(y/250)
        y = y.permute(0,3,2,1)
        y = y.permute(1,0,2,3)

        return x,y



# def getTransform():
#     return transforms.Compose([transforms.ToTensor()])

# transform = getTransform()

#데이터 로더에서 자료 가져올때 
BATCH_SIZE = 32

# train_dataset = TrainDataset(x_train_,y_train_,transform)
# valid_dataset = TrainDataset(x_val_,y_val_,transform)

train_dataset = TrainDataset(x_train_,y_train_)
valid_dataset = TrainDataset(x_val_,y_val_)


In [ ]:
# Dataloader 클래스는 데이터셋에서 배치 개수만큼 뽑아서 제공해줍니다
# 데이터 로더는 torch에서 제공해주는 기본 클래스를 사용했습니다.
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=2, prefetch_factor=2)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, worker_init_fn=2, prefetch_factor=2)


# 데이터 로더에서 가져오는 데이터의 모양 확인
for x, y in train_dataloader:
    print(x.shape, y.shape)
    break

    # (samples, time, rows, cols, channels)

In [ ]:
# convlstm 예제입니다.

# import torch.nn as nn
# import torch


# class ConvLSTMCell(nn.Module):

#     def __init__(self, input_dim, hidden_dim, kernel_size, bias):
#         """
#         Initialize ConvLSTM cell.

#         Parameters
#         ----------
#         input_dim: int
#             Number of channels of input tensor.
#         hidden_dim: int
#             Number of channels of hidden state.
#         kernel_size: (int, int)
#             Size of the convolutional kernel.
#         bias: bool
#             Whether or not to add the bias.
#         """

#         super(ConvLSTMCell, self).__init__()

#         self.input_dim = input_dim
#         self.hidden_dim = hidden_dim

#         self.kernel_size = kernel_size
#         self.padding = kernel_size[0] // 2, kernel_size[1] // 2
#         self.bias = bias

#         self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
#                               out_channels=4 * self.hidden_dim,
#                               kernel_size=self.kernel_size,
#                               padding=self.padding,
#                               bias=self.bias)

#     def forward(self, input_tensor, cur_state):
#         h_cur, c_cur = cur_state

#         combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

#         combined_conv = self.conv(combined)
#         cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
#         i = torch.sigmoid(cc_i)
#         f = torch.sigmoid(cc_f)
#         o = torch.sigmoid(cc_o)
#         g = torch.tanh(cc_g)

#         c_next = f * c_cur + i * g
#         h_next = o * torch.tanh(c_next)

#         return h_next, c_next

#     def init_hidden(self, batch_size, image_size):
#         height, width = image_size
#         return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
#                 torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))
        
        

In [ ]:
# pytorch convlstm 예제중 사용해보던 모델입니다.

# class EncoderDecoderConvLSTM(nn.Module):
#     def __init__(self, nf, in_chan):
#         super(EncoderDecoderConvLSTM, self).__init__()

#         """ ARCHITECTURE 

#         # Encoder (ConvLSTM)
#         # Encoder Vector (final hidden state of encoder)
#         # Decoder (ConvLSTM) - takes Encoder Vector as input
#         # Decoder (3D CNN) - produces regression predictions for our model

#         """
#         self.encoder_1_convlstm = ConvLSTMCell(input_dim=in_chan,
#                                                hidden_dim=nf,
#                                                kernel_size=(3, 3),
#                                                bias=True)

#         self.encoder_2_convlstm = ConvLSTMCell(input_dim=nf,
#                                                hidden_dim=nf,
#                                                kernel_size=(3, 3),
#                                                bias=True)

#         self.decoder_1_convlstm = ConvLSTMCell(input_dim=nf,  # nf + 1
#                                                hidden_dim=nf,
#                                                kernel_size=(3, 3),
#                                                bias=True)

#         self.decoder_2_convlstm = ConvLSTMCell(input_dim=nf,
#                                                hidden_dim=nf,
#                                                kernel_size=(3, 3),
#                                                bias=True)

#         self.decoder_CNN = nn.Conv3d(in_channels=nf,
#                                      out_channels=1,
#                                      kernel_size=(1, 3, 3),
#                                      padding=(0, 1, 1))


#     def autoencoder(self, x, seq_len, future_step, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4):

#         outputs = []

#         # encoder
#         for t in range(seq_len):
#             h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :],
#                                                cur_state=[h_t, c_t])  # we could concat to provide skip conn here
#             h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t,
#                                                  cur_state=[h_t2, c_t2])  # we could concat to provide skip conn here

#         # encoder_vector
#         encoder_vector = h_t2

#         # decoder
#         for t in range(future_step):
#             h_t3, c_t3 = self.decoder_1_convlstm(input_tensor=encoder_vector,
#                                                  cur_state=[h_t3, c_t3])  # we could concat to provide skip conn here
#             h_t4, c_t4 = self.decoder_2_convlstm(input_tensor=h_t3,
#                                                  cur_state=[h_t4, c_t4])  # we could concat to provide skip conn here
#             encoder_vector = h_t4
#             outputs += [h_t4]  # predictions

#         outputs = torch.stack(outputs, 1)
#         outputs = outputs.permute(0, 2, 1, 3, 4)
#         outputs = self.decoder_CNN(outputs)
#         outputs = torch.nn.Sigmoid()(outputs)

#         return outputs

#     def forward(self, x, future_seq=0, hidden_state=None):

#         """
#         Parameters
#         ----------
#         input_tensor:
#             5-D Tensor of shape (b, t, c, h, w)        #   batch, time, channel, height, width
#         """

#         # find size of different input dimensions
#         b, seq_len, _, h, w = x.size()

#         # initialize hidden states
#         h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
#         h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
#         h_t3, c_t3 = self.decoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
#         h_t4, c_t4 = self.decoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))

#         # autoencoder forward
#         outputs = self.autoencoder(x, seq_len, future_seq, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4)

#         return outputs

In [ ]:
# 공부하던 예제입니다.

# class test_Net(nn.Module): # nn.Module 모든 신경망 모듈의 기본이 되는 클래스
#                       # 각 층과 함수 등 신경망의 구성요소를 이 클래스 안에서 정의한다.
#                       # nn.Module은 모든 신경망 모듈의 기본이 되는 클래스로 레이어, 함수등을 정의하는구나!

#     def __init__(self):  # 초기화 함수   
#         super(test_Net, self).__init__()

#         self.convlstm1 = ConvLSTMCell(1, 64, (3,3),bias=True)  
#         self.batchnorm1 = nn.BatchNorm2d(256)
#         # self.convlstm2 = ConvLSTM(128,128,(3,3),1,bias=True)     
#         # self.batchnorm2 = nn.BatchNorm2d(1)
#         # self.convlstm3 = ConvLSTM(128,128,(3,3),1,bias=True)   
#         # self.batchnorm3 = nn.BatchNorm2d(1) 

#         self.conv4 = nn.Conv3d(3, 1, kernel_size=3, padding=1)

#     def forward(self, x):
#         x = self.convlstm1(x)
#         print(type(x))
#         print(len(x))

#         x = torch.tensor(x)
#         print(x.size())
#         x = self.batchnorm1(x)
#         # x = self.convlstm2(x)
#         # x = self.batchnorm2(x)        
#         # x = self.convlstm3(x)
#         # x = self.batchnorm3(x)
#         # print(x.size())
#         x = self.conv4(torch.tensor(x))

#         return x     

In [ ]:
# channels = 1
# model = ConvLSTM(input_dim=channels,
#                  hidden_dim=[64, 64, 128],
#                  kernel_size=(3, 3),
#                  num_layers=3,
#                  batch_first=True,
#                  bias=True,
#                  return_all_layers=False).to(device)
# model = EncoderDecoderConvLSTM().to(device)


#모델을 생성해주고 cuda 를 사용하게끔 설정해줍니다.
model = test_Net().to(device)
print(model)

In [ ]:
# Loss Function && etric Function
# 대회에서 제공해주던 산식코드입니다. 250으로 나눠준 상태입니다.

# metrics
def mae_score(true, pred):
    true, pred = numpy_to_tensor(true, pred)
    score = np.mean(np.abs(true-pred))
    
    return score

# metrics
def f1_score(true, pred):
    true, pred = numpy_to_tensor(true, pred)

    target = np.where((true>1*0.05)<1*0.5)
    
    true = true[target]
    pred = pred[target]
    true = np.where(true < 1*0.15, 0, 1)
    pred = np.where(pred < 1*0.15, 0, 1)
    
    
    right = np.sum(true * pred == 1)
    precision = right / np.sum(true+1e-8)
    recall = right / np.sum(pred+1e-8)
    score = 2 * precision*recall/(precision+recall+1e-8)
    
    return score
    
# loss function
def mae_over_f1(true, pred):
    mae = mae_score(true, pred)
    f1 = f1_score(true, pred)
    score = mae/(f1+1e-8)
    
    return score

def numpy_to_tensor(true, pred):

    return true.cpu().numpy().argmax(), pred.cpu().numpy().argmax()

In [ ]:
# Optimizer

# adam optimizer
# 옵티아미저 상세 설정 부분입니다.
opt_adam = optim.Adam(model.parameters(), lr=3e-4)

#현재 학습률을 나타내기위한 함수입니다.
def get_lr(opt):
    for param_group in opt.param_groups:
        return param_group["lr"]

# check our learning rate
current_lr = get_lr(opt_adam)
print(f"current_lr = {current_lr}")



# learning rate scheduler
# 학습에 도움을주는 스케쥴러 입니다.

lr_scheduler = ReduceLROnPlateau(opt_adam, mode="min", factor=0.5, patience=20, verbose=1)

# example 
# 학습에따른 학습률 감소를 보여줍니다.
for i in range(100):
    lr_scheduler.step(1)

In [ ]:
# Training 

def metrics_batch(pred, true, metrics):
    # if needed add param "metrics" to custom
    """
    output will be pred
    target will be corrects
    """
    if metrics:
        return list(map(lambda x: x(true, pred), metrics))
    mae_score = mae_score(true, pred)
    f1_score = f1_score(true, pred)
    return (mae_score, f1_score)

def loss_batch(loss_func, pred, true, opt=None):
    """
    loss_func => mae_over_f1
    """
    loss = loss_func(true, pred)
    with torch.no_grad():
        metrics = metrics_batch(pred, true, [mae_score, f1_score])
    if opt is not None:
        opt.zero_grad()
        # loss.backward()
        opt.step()  # 학습이 이뤄지는 곳
    return loss, metrics

def loss_epoch(model, loss_func, dataset_dataloader, sanity_check=False, opt=None):
    running_loss = 0.0
    running_metric = [0.0, 0.0]
    len_data = len(dataset_dataloader.dataset)

    for x, y in dataset_dataloader:
        x = x.to(device)
        y = y.to(device)
        # 모델 결과
        pred = model(x)
        # 손실함수 구하기
        loss, metrics = loss_batch(loss_func, pred, y, opt)
        # 손실함수 
        running_loss += loss
        if metrics is not None:
            for idx, metric_value in enumerate(metrics):
                running_metric[idx] += metric_value
        
        # 문제 있으면 break, 여기서는 True 일때 바로 break
        if sanity_check is True:
            break
    
    loss = running_loss / float(len_data)
    metrics = list(map(lambda x: x/float(len_data), metrics))
    print(loss, metrics)
    return loss, metrics

In [ ]:
# pytorch 기본코드에서 제작한 로스값 입니다.
loss_func = mae_over_f1

opt_adam = optim.Adam(model.parameters(), lr=3e-4)
# lr_scheduler = ReduceLROnPlateau(opt_adam, mode="min", factor=0.5, patience=20, verbose=1)

#학습에 필요한 파라미터들을 한번에 정리하고
#학습 메소드에 넣어줍니다.
TRAIN_PARAMS = {
    "num_epochs" : 10,
    "loss_func" : loss_func,
    "optimizer" : opt_adam,
    "train_dataloader" : train_dataloader,
    "valid_dataloader" : valid_dataloader,
    "sanity_check" : True,
    "lr_scheduler" : lr_scheduler,
    "save_path" : "./weights.pt"
}

In [ ]:
import copy

def train(model, params):
    num_epochs = params['num_epochs']
    loss_func = params['loss_func']
    opt = params["optimizer"]
    train_dataloader = params['train_dataloader']
    valid_dataloader = params['valid_dataloader']
    sanity_check = params['sanity_check']
    lr_scheduler = params['lr_scheduler']
    save_path = params['save_path']

    # keep history of the loss and metric
    loss_hist = {
        "train" : [],
        "valid" : []
    }

    metrics_hist = {
        "train" : [],
        "valid" : []
    }

    # copy best weights
    best_model_weights = copy.deepcopy(model.state_dict())
    # init best loss
    # 최저 로스값을 가져옵니다.
    best_loss = float("inf")

    #측정 할때는 반복문을 없애고 eval 로만 진행해야합니다.

    for epoch in range(num_epochs):
        current_lr = get_lr(opt)
        print(f'Epoch:{epoch}/{num_epochs-1}, current lr:{current_lr}')
        model.train()
        train_loss, train_metrics = loss_epoch(model, loss_func, train_dataloader, sanity_check, opt)

        # save history
        loss_hist["train"].append(train_loss)
        metrics_hist["train"].append(train_metrics)

        
        # model.eval()
        # with torch.no_grad():
    

    return model, loss_hist, metrics_hist

In [ ]:


my_model, loss_hist, metrics_hist = train(model, TRAIN_PARAMS)

print(loss_hist)